In the first exercise, you will design a small road network in SUMO according to specific requirements and then define a particular demand for the network. Using the SUMO config files, you will bring together the road network and traffic demand into a simulation. With the help of a Python script, you will start the simulation and extract macroscopic traffic information from it.

At the beginning, enter your name and student number in the next cell and execute it. Subsequently, your individual task will be generated.

The exercise is considered passed if at least 10 out of the 12 sub-tasks are successfully implemented.

In [1]:
from aufg import *

Name = 'Debanil Guha' 
Matrikelnummer = 157433

generate_aufgabe(Matrikelnummer)


    Your task in the first exercise will be to define a traffic network with 4 intersections/nodes (1),
    the edges between the nodes must have a length of 150 meters (2). All edges should be
    drivable in both directions, and have 2 lanes per direction (3). Within the
    traffic network, it is not possible to turn. This means that at the intersections it is not possible to turn
    left or right (4).

    The intersections are equipped with traffic lights for traffic control (5). Each edge must turn green once during the phase cycle
    and each phase is followed by a corresponding yellow phase and a red phase of at least 3
    seconds each (6). In total, each traffic light must have a cycle time of 120 seconds (7).

    Further define a traffic demand of a total of 50 vehicles. A vehicle should start every 2 seconds in the simulation (8). The vehicles should
    start alternately at the outer edges of the traffic network in a clockwise direction (9).

    Then enable the simula

### Check the Os Path that SUMO and its files are existing

In [2]:
import os
home_dir = os.path.expanduser("~")
zshrc_path = os.path.join(home_dir, ".zshrc")
bashrc_path = os.path.join(home_dir, ".bashrc")
def check_os_env_in_file(file_path):
    if os.path.exists(file_path):
        with open(file_path, "r") as f:
            content = f.read()
            if "SUMO_HOME" in content or "SUMO_GUI" in content:
                return True
    return False

zshrc_has_os = check_os_env_in_file(zshrc_path)

print(f"SUMO environment variables in .zshrc: {zshrc_has_os}")

SUMO environment variables in .zshrc: True


In [3]:
# Read environment variables from ~/.zshrc and set them in os.environ
if os.path.exists(zshrc_path):
    with open(zshrc_path, "r") as f:
        for line in f:
            if line.startswith("export "):
                parts = line.strip().split()
                if len(parts) == 2 and "=" in parts[1]:
                    key, value = parts[1].split("=", 1)
                    key = key.strip()
                    value = value.strip().strip('"').strip("'")
                    os.environ[key] = value
print("SUMO_HOME:", os.environ.get('SUMO_HOME'))
print("SUMO_GUI:", os.environ.get('SUMO_GUI'))

SUMO_HOME: /Users/debanilguha/sumo
SUMO_GUI: $SUMO_HOME/bin


In [4]:
!pip install traci

zsh:1: command not found: pip


In [ ]:
from sumolib import checkBinary
import os
import traci
import sys
import pandas as pd



if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("please declare environment variable 'SUMO_HOME'")


if 'SUMO_GUI' in os.environ:
    sumogui = os.path.join(os.environ['SUMO_HOME'], 'bin','sumo-gui')
else:
    sumogui = os.path.join('/Users/debanilguha/sumo', 'tools', 'sumo-gui')
    sys.exit("please declare environment variable 'SUMO_GUI'")



def run():
    edges = traci.edge.getIDList()
    densities = {e: 0 for e in edges}
    while traci.simulation.getTime() < 1000:
        traci.simulationStep()
        # Calculate density for each edge
        for edge in edges:
            cars_on_edge = traci.edge.getLastStepVehicleNumber(edge)
            edge_length = traci.lane.getLength(edge+'_0')  # Get actual edge length
            density = cars_on_edge / edge_length if edge_length > 0 else 0  # cars per meter
            densities[edge] += density
    print("Waiting for simulation to finish...Close the GUI to stop it.")
    traci.close()
    print("Simulation finished.","\nDensities per edge:")
    # Convert densities dictionary to DataFrame
    df_density = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in densities.items()]))
    # Density based
    for e in edges:
        print(f"Density on edge {e}: {densities[e]:.2f} cars/m")

if __name__ == "__main__":
    sumoBinary = checkBinary(sumogui)
    traci.start([sumoBinary, "-c", "./sumo_conf_files/network.sumocfg"]) #Restart Kernel if you get an error here
        
    run()

 Retrying in 1 seconds
Junctions in the network: ['J1', 'J2', 'J3', 'J4']
Waiting for simulation to finish...Close the GUI to stop it.
Simulation finished. 
Densities per edge:
Density on edge :J1_0: 22.78 cars/m
Density on edge :J1_1: 34.58 cars/m
Density on edge :J1_2: 21.25 cars/m
Density on edge :J1_3: 26.46 cars/m
Density on edge :J2_0: 36.11 cars/m
Density on edge :J2_1: 27.92 cars/m
Density on edge :J2_2: 9.86 cars/m
Density on edge :J2_3: 35.35 cars/m
Density on edge :J3_0: 35.14 cars/m
Density on edge :J3_1: 9.79 cars/m
Density on edge :J3_2: 18.06 cars/m
Density on edge :J3_3: 22.15 cars/m
Density on edge :J4_0: 16.39 cars/m
Density on edge :J4_1: 17.85 cars/m
Density on edge :J4_2: 14.31 cars/m
Density on edge :J4_3: 10.35 cars/m
Density on edge E1_2: 59.36 cars/m
Density on edge E1_3: 73.64 cars/m
Density on edge E2_1: 82.16 cars/m
Density on edge E2_4: 26.81 cars/m
Density on edge E3_1: 44.42 cars/m
Density on edge E3_4: 11.06 cars/m
Density on edge E4_2: 14.02 cars/m
Dens